# What we are doing
- We are following a portion of a tutorial outlined in this paper: https://doi.org/10.1038/s41596-020-0353-1.
- The portion we are following is QC of base data.
- We will take a pre-QC data file called `Height.gwas.txt.gz` and perform some QC described by the paper.
- After performing QC, we will check our result against a correctly QC'd file called `Height.QC.gz`.

# Requirements
- Download `Height.gwas.txt.gz` from [this link](https://drive.google.com/file/d/1RWjk49QNZj9zvJHc9X_wyZ51fdy6xQjv/view?usp=sharing), and place it in the same directory as this notebook.
- Download `post-qc.zip` from [this link](https://drive.google.com/file/d/1x_G0Gxk9jFMY-PMqwtg6-vdEyUPp5p5u/view), unzip it and copy `Height.QC.gz` into the same directory as this notebook.

In [1]:
import numpy as np
import pandas as pd

In [2]:
base = pd.read_csv('Height.gwas.txt.gz', sep='\t', compression='gzip')
base.head()

,CHR,BP,SNP,A1,A2,N,SE,P,OR,INFO,MAF
0,1,756604,rs3131962,A,G,388028,0.003017,0.483171,0.997887,0.890558,0.369390
1,1,768448,rs12562034,A,G,388028,0.003295,0.834808,1.000687,0.895894,0.336846
2,1,779322,rs4040617,G,A,388028,0.003033,0.428970,0.997604,0.897508,0.377368
3,1,801536,rs79373928,G,T,388028,0.008413,0.808999,1.002036,0.908963,0.483212
4,1,808631,rs11240779,G,A,388028,0.002428,0.590265,1.001308,0.893213,0.450410


# QC checklist: Base data

## File transfer
A common problem is that the downloaded base data file can be corrupted during download, which can cause PRS software to crash or to produce errors in results. However, a md5sum hash is generally included in files so that file integrity can be checked.

If the file `Height.gwas.txt.gz` is intact, then md5sum generates a string of characters, which in this case should be: `a2b15fb6a2bbbe7ef49f67959b43b160`

In [3]:
%%bash --out checksum
# IPython magic that saves bash output to a Python variable 'checksum'

md5sum Height.gwas.txt.gz

In [4]:
# Outputs 'True' if hashes match, which means data in not corrupted
checksum.split()[0] == 'a2b15fb6a2bbbe7ef49f67959b43b160'

True

## Heritability check
It is recommended that PRS analyses are performed on base data with a chip-heritability estimate of $h_{snp}^2 > 0.05$

In [5]:
# Height GWAS data is simulated to have a chip-heritability much greater than 0.05.
# Nothing to do.

## Effect allele
Clarify which data column has the effect allele.

In [6]:
# Note that column 'A1' holds the effect allele.

## Genomic build
Base data and target data should be generated from the same genome build.  
Tools such as [LiftOver](https://genome.ucsc.edu/cgi-bin/hgLiftOver) can be used to make builds consistent across data sets.

In [7]:
# Both base and target data is based on same genome build, according to the data provider.
# Nothing to do.

## Standard GWAS QC
SNPs with low minor allele frequency (MAF) or imputation information score (INFO) are more likely to generate false positive results due to their lower statistical power (and higher probability of genotyping errors in the case of low MAF). Therefore, SNPs with low MAF or low INFO are typically removed before performing downstream analyses.

The paper recommends removing SNPs with MAF < 1% or INFO < 0.8 (with very large base sample sizes these thresholds could be reduced if sensitivity checks indicate reliable results).

In [8]:
low_stat_power = (base['MAF'] < 0.01) | (base['INFO'] < 0.8)
base_qc = base[~low_stat_power]

print('Number of low statistical power SNPs removed: {}'.format(low_stat_power.sum()))

Number of low statistical power SNPs removed: 9


## Duplicate SNPs
If an error has occurred in the generation of the base data then there may be duplicated SNPs in the base data file. Most PRS software do not allow duplicated SNPs in the base data input and thus they should be removed.

In [9]:
dupes = base_qc.duplicated(keep='first')
base_qc = base_qc[~dupes]

print('Number of duplicated SNPs removed: {}'.format(dupes.sum()))

Number of duplicated SNPs removed: 2


## Ambiguous SNPs
If the base and target data were generated using different genotyping chips and the chromosome strand (+/-) that was used for either is unknown, then it is not possible to pair-up the alleles of ambiguous SNPs (i.e. those with complementary alleles, either C/G or A/T SNPs) across the data sets, because it will be unknown whether the base and target data are referring to the same allele or not. While allele frequencies could be used to infer which alleles are on the same strand, the accuracy of this could be low for SNPs with MAF close to 50% or when the base and target data are from different populations. Therefore, we recommend removing all ambiguous SNPs to avoid introducing this potential source of systematic error.

Ambiguous SNPs can be removed in the base data and then there will be no such SNPs in the subsequent analyses, since analyses are performed only on SNPs that overlap between the base and target data.

In [10]:
ambiguous = ((base_qc['A1'] == 'A') & (base_qc['A2'] == 'T')) | \
            ((base_qc['A1'] == 'T') & (base_qc['A2'] == 'A')) | \
            ((base_qc['A1'] == 'C') & (base_qc['A2'] == 'G')) | \
            ((base_qc['A1'] == 'G') & (base_qc['A2'] == 'C'))

base_qc = base_qc[~ambiguous]

print('Number of ambiguous SNPs removed: {}'.format(ambiguous.sum()))
print('Number of non-ambiguous SNPs remaining: {}'.format(len(base_qc)))

Number of ambiguous SNPs removed: 29876
Number of non-ambiguous SNPs remaining: 499617


# Checking our QC against the provided QC
If we performed the QC correctly, it should match the QC provided called `Height.QC.gz`

In [11]:
base_qc.reset_index(drop=True, inplace=True)  # reset index before comparing

answer = pd.read_csv('Height.QC.gz', sep='\t', compression='gzip')

for col in base_qc.columns:
    # Allow tolerance when comparing columns with floats
    if (answer[col].dtype == np.float64) or \
       (answer[col].dtype == np.float32) or \
       (answer[col].dtype == np.float16):
        col_match = np.allclose(base_qc[col], answer[col], atol=1e-8)
    else:
        col_match = all(base_qc[col] == answer[col])
        
    print('Column "{}" match result:\t{}'.format(col, col_match))

Column "CHR" match result:	True
Column "BP" match result:	True
Column "SNP" match result:	True
Column "A1" match result:	True
Column "A2" match result:	True
Column "N" match result:	True
Column "SE" match result:	True
Column "P" match result:	True
Column "OR" match result:	True
Column "INFO" match result:	True
Column "MAF" match result:	True


All good! QC performed successfully!